In [23]:
%load_ext autoreload
%autoreload 2

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.utils.data import DataLoader
# import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import random
from itertools import cycle
from dotenv import load_dotenv
import os
import tqdm.notebook as tqdm
# from peft import PeftModel, PeftConfig, get_peft_model
# from datasets import load_dataset
# import wandb
from tqdm import tqdm
import json

load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")




from dataclasses import asdict
from datetime import datetime


exp_time = datetime.now().strftime("%b%d")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B-int4"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
).eval()

In [ ]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)

save_dir = "/root/multimodal-eng/models/3jun-int4"
model.save_pretrained(save_dir)
checkpoint = "/root/multimodal-eng/models/3jun-int4"
# model = load_checkpoint_and_dispatch(model, MODEL_PATH, device_map=device_map, dtype=TORCH_TYPE)
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()

In [ ]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
).to(DEVICE).eval()

text_only_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {} ASSISTANT:"

while True:
    image_path = input("image path >>>>> ")
    if image_path == '':
        print('You did not enter image path, the following will be a plain text conversation.')
        image = None
        text_only_first_query = True
    else:
        image = Image.open(image_path).convert('RGB')

    history = []

    while True:
        query = input("Human:")
        if query == "clear":
            break

        if image is None:
            if text_only_first_query:
                query = text_only_template.format(query)
                text_only_first_query = False
            else:
                old_prompt = ''
                for _, (old_query, response) in enumerate(history):
                    old_prompt += old_query + " " + response + "\n"
                query = old_prompt + "USER: {} ASSISTANT:".format(query)
        if image is None:
            input_by_model = model.build_conversation_input_ids(
                tokenizer,
                query=query,
                history=history,
                template_version='chat'
            )
        else:
            input_by_model = model.build_conversation_input_ids(
                tokenizer,
                query=query,
                history=history,
                images=[image],
                template_version='chat'
            )
        inputs = {
            'input_ids': input_by_model['input_ids'].unsqueeze(0).to(DEVICE),
            'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(DEVICE),
            'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(DEVICE),
            'images': [[input_by_model['images'][0].to(DEVICE).to(TORCH_TYPE)]] if image is not None else None,
        }
        gen_kwargs = {
            "max_new_tokens": 2048,
            "pad_token_id": 128002,  
        }
        with torch.no_grad():
            outputs = model.generate(**inputs, **gen_kwargs)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            response = tokenizer.decode(outputs[0])
            response = response.split("<|end_of_text|>")[0]
            print("\nCogVLM2:", response)
        history.append((query, response))


# getting batched inference

In [ ]:
"""
This is a demo for using CogVLM2 in CLI using multi-GPU with lower memory.
If your single GPU is not enough to drive this model, you can use this demo to run this model on multiple graphics cards with limited video memory.
Here, we default that your graphics card has 24GB of video memory, which is not enough to load the FP16 / BF16 model.
so , need to use two graphics cards to load. We set '23GiB' for each GPU to avoid out of memory.
"""
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import textwrap

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)
checkpoint = "/root/multimodal-eng/models/28may-1"
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()
print("Model loaded.")

text_only_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {} ASSISTANT:"

image_folder = "/root/multimodal-eng/pics_of_mates_faces/"
user_input = "Describe this person to me."

images_and_responses = []


In [ ]:
batched_inputs = []

for idx, image_file in tqdm(enumerate(os.listdir(image_folder))):
    if not image_file.endswith(".png"):
        continue
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path).convert('RGB')

    history = []

    input_by_model = model.build_conversation_input_ids(
        tokenizer,
        query=user_input,
        images=[image],
        template_version='chat'
    )
    inputs = {
        'input_ids': input_by_model['input_ids'].unsqueeze(0).to(DEVICE),
        'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(DEVICE),
        'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(DEVICE),
        'images': [[input_by_model['images'][0].to(DEVICE).to(TORCH_TYPE)]] if image is not None else None,
    }

    batched_inputs.append(inputs)

batched_input_ids = torch.cat([inputs['input_ids'] for inputs in batched_inputs], dim=0)


In [ ]:
batched_input_ids.shape

In [ ]:
with torch.no_grad():
    outputs = model.generate(
        input_ids=batched_input_ids[:4, :],
        token_type_ids=torch.cat([inputs['token_type_ids'] for inputs in batched_inputs[:4]], dim=0),
        max_new_tokens=2048,
        pad_token_id=128002,
        images=[inputs['images'][0] for inputs in batched_inputs[:4]]
    )
for output_idx, outputs in enumerate(outputs):
    response = tokenizer.decode(outputs)
    response = response.split("<|end_of_text|>")[0].split("Question:")[-1]
    print(response)

# Jsonformer

In [ ]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

car = {
  "type": "object",
  "properties": {
    "car": {
      "type": "object",
      "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "number"},
        "colors": {
          "type": "array",
          "items": {"type": "string"}
        },
        "features": {
          "type": "object",
          "properties": {
            "audio": {
              "type": "object",
              "properties": {
                "brand": {"type": "string"},
                "speakers": {"type": "number"},
                "hasBluetooth": {"type": "boolean"}
              }
            },
            "safety": {
              "type": "object",
              "properties": {
                "airbags": {"type": "number"},
                "parkingSensors": {"type": "boolean"},
                "laneAssist": {"type": "boolean"}
              }
            },
            "performance": {
              "type": "object",
              "properties": {
                "engine": {"type": "string"},
                "horsepower": {"type": "number"},
                "topSpeed": {"type": "number"}
              }
            }
          }
        }
      }
    },
    "owner": {
      "type": "object",
      "properties": {
        "firstName": {"type": "string"},
        "lastName": {"type": "string"},
        "age": {"type": "number"},
      }
    }
  }
}

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="Generate an example car",
)

print("Generating...")
output = builder()

highlight_values(output)


In [ ]:
tokenizer.paddding_side = "left"

while True:
    text_input = input("Enter a question: ")
    input1 = "Question: " + text_input + "Short Answer:"
    input2 = "Question: " + text_input + "Long Answer:"
    if text_input == "exit":
        break
    batch_tokens = tokenizer([input1, input2], return_tensors="pt")
    out = model.generate(**batch_tokens)
    print(f"Short Answer: {tokenizer.decode(out[0])}")
    print(f"Long Answer: {tokenizer.decode(out[1])}")



In [ ]:
model.cpu()

In [ ]:
"""
This is a demo for using CogVLM2 in CLI using multi-GPU with lower memory.
If your single GPU is not enough to drive this model, you can use this demo to run this model on multiple graphics cards with limited video memory.
Here, we default that your graphics card has 24GB of video memory, which is not enough to load the FP16 / BF16 model.
so , need to use two graphics cards to load. We set '23GiB' for each GPU to avoid out of memory.
"""
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import textwrap

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)
checkpoint = "/root/multimodal-eng/models/28may-1"
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()
print("Model loaded.")

In [ ]:

car = {
  "type": "object",
  "properties": {
    "car": {
      "type": "object",
      "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "number"},
        "colors": {
          "type": "array",
          "items": {"type": "string"}
        },
        "features": {
          "type": "object",
          "properties": {
            "audio": {
              "type": "object",
              "properties": {
                "brand": {"type": "string"},
                "speakers": {"type": "number"},
                "hasBluetooth": {"type": "boolean"}
              }
            },
            "safety": {
              "type": "object",
              "properties": {
                "airbags": {"type": "number"},
                "parkingSensors": {"type": "boolean"},
                "laneAssist": {"type": "boolean"}
              }
            },
            "performance": {
              "type": "object",
              "properties": {
                "engine": {"type": "string"},
                "horsepower": {"type": "number"},
                "topSpeed": {"type": "number"}
              }
            }
          }
        }
      }
    },
    "owner": {
      "type": "object",
      "properties": {
        "firstName": {"type": "string"},
        "lastName": {"type": "string"},
        "age": {"type": "number"},
      }
    }
  }
}

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="Generate an example car",
)

print("Generating...")
output = builder()

highlight_values(output)


# CogVLMJsonformer

In [1]:
"""
This is a demo for using CogVLM2 in CLI using multi-GPU with lower memory.
If your single GPU is not enough to drive this model, you can use this demo to run this model on multiple graphics cards with limited video memory.
Here, we default that your graphics card has 24GB of video memory, which is not enough to load the FP16 / BF16 model.
so , need to use two graphics cards to load. We set '23GiB' for each GPU to avoid out of memory.
"""
%load_ext autoreload
%autoreload 2
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import textwrap
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

from transformers import AutoModelForCausalLM, AutoTokenizer

from subclass_jsonformer import CogVLMJsonformer



car = {
  "type": "object",
  "properties": {
    "car": {
      "type": "object",
      "properties": {
        "make": {"type": "string"},
        "model": {"type": "string"},
        "year": {"type": "number"},
        "colors": {
          "type": "array",
          "items": {"type": "string"}
        },
        "features": {
          "type": "object",
          "properties": {
            "audio": {
              "type": "object",
              "properties": {
                "brand": {"type": "string"},
                "speakers": {"type": "number"},
                "hasBluetooth": {"type": "boolean"}
              }
            },
            "safety": {
              "type": "object",
              "properties": {
                "airbags": {"type": "number"},
                "parkingSensors": {"type": "boolean"},
                "laneAssist": {"type": "boolean"}
              }
            },
            "performance": {
              "type": "object",
              "properties": {
                "engine": {"type": "string"},
                "horsepower": {"type": "number"},
                "topSpeed": {"type": "number"}
              }
            }
          }
        }
      }
    },
    "owner": {
      "type": "object",
      "properties": {
        "firstName": {"type": "string"},
        "lastName": {"type": "string"},
        "age": {"type": "number"},
      }
    }
  }
}

/root/.venv/multi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)
checkpoint = "/root/multimodal-eng/models/28may-1"
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()
print("Model loaded.")


# builder = Jsonformer(


In [2]:
MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B-int4"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:

from PIL import Image
image = Image.open("/root/multimodal-eng/pics_of_mates_faces/Screenshot_2020_04_06_at_151459__.png").convert('RGB')

builder = CogVLMJsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car,
    prompt="Generate an example car",
    images=[image],
    debug=True,
)

print("Generating...")
output = builder()

highlight_values(output)


In [17]:
prompt = "- problems: numbers suck, speed is slow, need batching with images and language stuff, OOMs for image inputs with the 4 bit model, but in principle I can run inference on this model with one gpu, maybe I should debug with this model on the 80gb a100, except even on an a100 I get errors! What happens when I only run one inference step with an image text pair, I might as well get super familiar with how to do this. "*10
prompt12 = "Describe this person in a poem"
prompt_empty = ""

image = Image.open("/root/multimodal-eng/pics_of_mates_faces/Screenshot_2020_04_06_at_151459__.png").convert('RGB')
tokenizer.padding_side = "left"

empty_build_input_ids = model.build_conversation_input_ids(
    tokenizer=tokenizer,
    query=prompt_empty,
    images=[image] if image is not None else None,
    template_version="base",
)
prompt_build_input_ids = model.build_conversation_input_ids(
    tokenizer=tokenizer,
    query=prompt_empty + prompt12,
    images=[image] if image is not None else None,
    template_version="base",
)
input_tokens = empty_build_input_ids['input_ids'].to(model.device)
image_only_inputs1 = {
    'input_ids': empty_build_input_ids['input_ids'].unsqueeze(0).to(model.device),
    'token_type_ids': empty_build_input_ids['token_type_ids'].unsqueeze(0).to(model.device),
    'attention_mask': empty_build_input_ids['attention_mask'].unsqueeze(0).to(model.device),
    'images': [[empty_build_input_ids['images'][0].to(model.device).to(torch.bfloat16)]] if image is not None else None,
}
image_only_inputs2 = {
    "input_ids": torch.stack([empty_build_input_ids['input_ids'], empty_build_input_ids['input_ids']], dim=0).to(model.device),
    "token_type_ids": torch.stack([empty_build_input_ids['token_type_ids'], empty_build_input_ids['token_type_ids']], dim=0).to(model.device),
    "attention_mask": torch.stack([empty_build_input_ids['attention_mask'], empty_build_input_ids['attention_mask']], dim=0).to(model.device),
    "images": [[empty_build_input_ids['images'][0].to(model.device).to(torch.bfloat16)]]*2 if image is not None else None,
}
prompt_only_inputs1 = {
    'input_ids': prompt_build_input_ids['input_ids'].unsqueeze(0).to(model.device),
    'token_type_ids': prompt_build_input_ids['token_type_ids'].unsqueeze(0).to(model.device),
    'attention_mask': prompt_build_input_ids['attention_mask'].unsqueeze(0).to(model.device),
    'images': [[prompt_build_input_ids['images'][0].to(model.device).to(torch.bfloat16)]] if image is not None else None,
}
prompt_only_inputs2 = {
    "input_ids": torch.stack([prompt_build_input_ids['input_ids'], prompt_build_input_ids['input_ids']], dim=0).to(model.device),
    "token_type_ids": torch.stack([prompt_build_input_ids['token_type_ids'], prompt_build_input_ids['token_type_ids']], dim=0).to(model.device),
    "attention_mask": torch.stack([prompt_build_input_ids['attention_mask'], prompt_build_input_ids['attention_mask']], dim=0).to(model.device),
    "images": [[prompt_build_input_ids['images'][0].to(model.device).to(torch.bfloat16)]]*2 if image is not None else None,
}
gen_kwargs = {
    "max_new_tokens": 30,
    # "temperature": 0,
    # "pad_token_id": tokenizer.eos_token_id,
    # "logits_processor": [number_logit_processor],
    "do_sample": False,
    # "num_return_sequences": 1,
    # "stopping_criteria": [
    #     StringStoppingCriteria(tokenizer, len(input_tokens))
    # ],
}

with torch.no_grad():
    output = model.forward(**image_only_inputs1, use_cache=True)
# generation_output = model.generate(**image_only_inputs2, **gen_kwargs, return_dict_in_generate=True)
# with torch.no_grad():
#     output = model.forward(**image_only_inputs, use_cache=True, past_key_values=None)
# response = model.generate(**inputs2, **gen_kwargs)
response_kv = model.generate(**prompt_only_inputs2, **gen_kwargs, past_key_values=output.past_key_values)
# output = model(**inputs)
# response_strs = tokenizer.decode(response, skip_special_tokens=True)
# logits = output.logits[0, -1]
torch.cuda.empty_cache()



/root/.venv/multi/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.venv/multi/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [28]:
response_kv.shape

torch.Size([1, 2314])

In [30]:
tokenizer.decode(response_kv[0], skip_special_tokens=True)



'Describe this person in a poem'

In [25]:
dtype = torch.bfloat16
llama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=hf_access_token, torch_dtype=dtype).cuda()
# llama = AutoModelForCausalLM.from_pretrained("models/now", token=hf_access_token, torch_dtype=dtype).cuda()
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# tokenizer.pad_token_id = tokenizer.unk_token_id
# tokenizer.padding_side = "left"
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
llama_tokenizer.pad_token = llama_tokenizer.eos_token


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.67s/it]


In [35]:
# Generation as usual
prompt = "Question: Please write a function in Python that transforms bytes to Giga bytes.\n\nAnswer: Here"
llama_inputs = llama_tokenizer(prompt, return_tensors='pt').to(llama.device)
generation_output = llama.generate(**llama_inputs, max_new_tokens=6, return_dict_in_generate=True)
decoded_output = llama_tokenizer.batch_decode(generation_output.sequences)[0]

# Piping the returned `past_key_values` to speed up the next conversation round
prompt = decoded_output + "\nQuestion: How can I modify the function above to return Mega bytes instead?\n\nAnswer: Here"
llama_inputs = llama_tokenizer(prompt, return_tensors='pt').to(llama.device)
generation_output = llama.generate(
  **llama_inputs,
  past_key_values=generation_output.past_key_values,
  max_new_tokens=6,
  return_dict_in_generate=True
)
llama_tokenizer.batch_decode(generation_output.sequences)[0][len(prompt):]

'Here is how you can modify the'